In [1]:
import json
import boto3
import psycopg2
from psycopg2 import extras
from nba_api.stats.endpoints import leaguegamefinder, boxscoretraditionalv2
from nba_api.stats.static import teams, players
import os
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import logging
import time

# Configure logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

def lambda_handler(event, context):
    """
    AWS Lambda handler function to fetch NBA team, player, and game data, process it,
    and store it in Amazon RDS.
    
    Parameters:
    - event (dict): Event data that triggers the Lambda function.
    - context (object): Provides runtime information to the handler.
    
    Returns:
    - dict: Status message indicating success or failure.
    """
    
    try:
        # Determine if this is an initial load or incremental update
        is_initial_load = event.get('initial_load', False)
        
        if is_initial_load:
            # Fetch and store static data
            teams_df = get_teams()
            print(teams_df)
            # store_teams_in_rds(teams_df)
            logger.info(f'Teams data: {len(teams_df)} records after processing.')
            
            players_df = get_players()
            print(players_df)
            # store_players_in_rds(players_df)
            logger.info(f'Players data: {len(players_df)} records after prcoessing.')
            logger.info("Static data (Teams and Players) stored successfully.")
            
            # Set start_date for game logs
            start_date = datetime(2024,10,1)    # Season start
        else:
            start_date = datetime.now() - timedelta(days=1)    # Yesterday
        
        # Determine the date for game fetching
        end_date = datetime.now() - timedelta(days=1)    # Yesterday
        start_date_str = start_date.strftime('%m/%d/%Y')
        end_date_str = end_date.strftime('%m/%d/%Y')
        logger.info(f"Fetching NBA data from {start_date_str} - {end_date_str}")
        
        # Fetch game data as a Pandas DataFrame
        game_logs_df = get_game_logs(start_date_str, end_date_str)
        if not game_logs_df.empty:
            game_logs_df = process_game_logs(game_logs_df)
            print(game_logs_df)
            logger.info(f"Retrieved {len(game_logs_df)} games from NBA API")
            logger.info(f"Processed games data: {len(game_logs_df)} records after filtering.")

    #         # Store the processed data into Amazon RDS
    #         store_game_stats_in_rds(game_stats_df)
    #         store_games_in_rds(games_df)
    #         logger.info("Game logs data successfully stored in Amazon RDS")

            # Fetch and store boxscores for each game
            boxscore_list = []
            for game_id in game_logs_df['game_id'].unique():
                try:
                    boxscore_df = get_boxscores(game_id)
                    boxscores_list.append(boxscore_df)
                    logger.info(f"Box scores for game_id {game_id} stored successfully")
                except Exception as box_e:
                    logger.error(f"Failed to process box scores for game_id {game_id}: {str(box_e)}")
                    continue    # Continue processing games
            boxscores_df = pd.concat(boxscore_list, ignore_index=True) if boxscore_list else pd.DataFrame()
            print(boxscores_df)
            if not boxscores_df.empty:
                clean_boxscores_df = process_boxscores(boxscores_df)
                # store_boxscores_in_rds(clean_boxscores_df)
            else:
                logger.info('No boxscores found.')
        else:
            logger.info('No games found for the specified date range.')
        
        return {
            'statusCode': 200,
            'body': json.dumps('Data ingestion complete and stored successfully.')
        }
    
    except Exception as e:
        logger.error(f"Error in lambda_handler: {str(e)}")
        return {
            'statusCode': 500,
            'body': json.dumps(f"Data ingestion failed: {str(e)}")
        }
    
def get_teams():
    """
    Fetches static team data from nba_api and returns it as a DataFrame.
    
    Parameters:
    - None
    
    Returns:
    - pd.DataFrame: Pandas dataframe comtaining team data.
    """
    
    try:
        # Fetch teams data
        nba_teams = teams.get_teams()
        teams_df = pd.DataFrame(nba_teams)
        
        # Process data
        teams_df = process_teams(teams_df)
        logger.info(f"Fetched and processed {len(teams_df)} teams from nba_api")
        
        return teams_df
    
    except Exception as e:
        logger.error(f"Error in fetch_and_store_teams: {str(e)}")
        raise e
        
def process_teams(teams_df):
    """
    Processes static team data and returns the processed data as a dataframe.
    
    Parameters:
    - teams_df (pd.DataFrame): A Pandas dataframe containing raw team data.
    
    Returns:
    - pd.DataFrame: A Pandas dataframe contaning processed team data.
    """
    
    try:
        required_cols = {'id', 'full_name', 'nickname', 'abbreviation', 'city', 'state', 'year_founded'}
        missing = required_cols - set(teams_df.columns)
        if missing:
            logger.warning(f"Missing expected columns: {missing}")
        
        processed_df = teams_df[['id', 'full_name', 'nickname', 'abbreviation', 'city', 'state', 'year_founded']]
        processed_df.rename(columns={
            'id': 'team_id',
            'abbreviation': 'team_abbreviation',
            'full_name': 'team_name'
        }, inplace=True)
        
        # Logging
        logger.info(f'Processed team data.')
        
        return processed_df
    
    except Exception as e:
        logger.error(f'Error in process_teams: {str(e)}')
        raise e
    
        
def get_players():
    """
    Fetches static player data from nba_api and returns it as a Pandas dataframe.
    
    Parameters:
    - None
    
    Returns:
    - pd.DataFrame: a Pandass dataframe containing player data.
    """
    
    try:
        # Fetch players data
        nba_players = players.get_players()
        players_df = pd.DataFrame(nba_players)
        
        # Process player data
        processed_players_df = process_players(players_df)
        logger.info(f"Fetched {len(players_df)} players from nba_api")
        
        return processed_players_df
    
    except Exception as e:
        logger.error(f"Error in fetch_and_store_players: {str(e)}")
        raise e
        
def process_players(players_df):
    """
    Processes static player data and returns the processed data as a Pandas dataframe.
    
    Paramters:
    - players_df (pd.DataFrame): A Pandas dataframe containing raw player data.
    
    Returns:
    - pd.DataFrame: A Pandas dataframe containing processed player data.
    """
    
    try:
        required_cols = {'is_active', 'id', 'full_name'}
        missing = required_cols - set(players_df.columns)
        if missing:
            logger.warning(f"Missing expected columns: {missing}")

        df_processed = players_df[players_df['is_active'] == True]    # Get active players
        df_processed.drop('is_active', axis=1, errors='ignore', inplace=True)
        
        df_processed.rename(columns={
            'id': 'player_id',
            'full_name': 'player_name'
        }, inplace=True)
        
        # Logging
        logger.info('Processed player data.')
        
        return df_processed
    
    except Exception as e:
        logger.error(f'Error in process_players: {str(e)}')
        raise e
        
def get_game_logs(start_date_str, end_date_str, max_retries=3, initial_wait=2):
    """
    Fetches game logs over a specified period from nba-api.
    
    Paramters:
    - start_date (str): The first date in the range
    - end_date (str): The last day in the range
    - max_retries (int): Maximum number of retry attempts.
    - initial_wait (int): Initial wait time (in seconds) before the first retry.
    
    Returns:
    - pd.DataFrame: A Pandas dataframe containing game logs from the specified period.
    """
    
    # Initialize the NBA Game Finder
    gamefinder = leaguegamefinder.LeagueGameFinder(
        league_id_nullable='00',            # '00' corresponds to the NBA
        season_nullable='2024-25',          # Get current season
        date_from_nullable=start_date_str,
        date_to_nullable=end_date_str
    )
    
    for attempt in range(max_retries):
        try:
            game_logs_df = gamefinder.get_data_frames()[0]
            
            return game_logs_df
        
        except Exception as e:
            logger.error(f'Error in get_game_logs: {str(e)}')
            
            # Retry until max retries
            if attempt < max_retries - 1:
                wait_time = initial_wait * (2 ** attempt)  # Exponential backoff
                logger.info(f"Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
            else:
                # All attempts have failed
                raise RuntimeError(f"Failed to fetch game logs after {max_retries} attempts.")
        
def process_game_logs(df):
    """
    Processes raw boxscore data and returns the processed data as a Pandas dataframe.
    
    Paramters:
    - pd.DataFrame: a Pandas dataframe containing raw box score data.
    
    Returns:
    - pd.DataFrame: A Pandas dataframe containing processed player data.
    """
    
    try:
        # Fix column structure
        processed_df = df.drop(['TEAM_ABBREVIATION', 'TEAM_NAME', 'FG_PCT', 'FT_PCT', 'FG3_PCT'], axis=1, errors='ignore')    # Drop unneccary columns
        processed_df.columns = processed_df.columns.str.lower()    # Lowercase columns
        
        # Fix datatypes
        processed_df['season_id'] = pd.to_numeric(processed_df['season_id'], errors='coerce').fillna(0).astype('int64')
        processed_df['game_date'] = pd.to_datetime(processed_df['game_date'])
        processed_df['season_id'] = pd.to_numeric(processed_df['season_id'], errors='coerce').fillna(0).astype('int64')
        
        return processed_df
    
    except Exception as e:
        logger.error(f'Error in process_game_logs: {str(e)}')
        raise e
        
def get_boxscores(game_id, max_retries=3, initial_wait=2):
    """
    Fetches box score data for a specific game.
    
    Parameters:
    - game_id (str): The unique identifier for the game.
    - max_retries (int): Maximum number of retry attempts.
    - initial_wait (int): Initial wait time in seconds before the first retry.
    
    Returns:
    - pd.DataFrame: A Pandas DataFrame containing the boxscore for the game.
    """
    
    for attempt in range(max_retries):
        try:
            boxscore = boxscoretraditionalv2.BoxScoreTraditionalV2(game_id=game_id)
            boxscore_df = boxscore.get_data_frames()[0]
            logger.info(f"Fetched box scores for game_id: {game_id} on attempt {attempt+1}")
            return boxscore_df
        except Exception as e:
            logger.error(f"Error fetching box scores for game_id {game_id} (attempt {attempt+1}/{max_retries}): {str(e)}")
            
            # Retry 
            if attempt < max_retries - 1:
                wait_time = initial_wait * (2 ** attempt)  # Exponential backoff: 2s, 4s, 8s, ...
                logger.info(f"Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
            else:
                raise RuntimeError(f"Failed to fetch box scores for game_id {game_id} after {max_retries} attempts.")
    
def process_boxscores(boxscores_df):
    """
    Processes boxscore data to prepare for RDS insertion.
    
    Paramters:
    - boxscores_df (pd.DataFrame): A Pandas dataframe containing boxscore data.
    
    Returns:
    - pd.DataFrame: A Pandas dataframe contining processed boxscore data.
    """
    
    try:
        # Fix column structure
        clean_boxscores_df = boxscores_df.drop(['TEAM_ABBREVIATION', 'TEAM_CITY', 'PLAYER_NAME', 'NICKNAME', 'FG_PCT', 'FG3_PCT', 'FT_PCT'], axis=1, errors='ignore')
        clean_boxscores_df.columns = clean_boxscores_df.columns.str.lower()
        
        # Fix datatypes
        cols_to_convert_to_int64 = [
            'fgm', 'fga', 'fg3m', 'fg3a', 'ftm', 'fta', 
            'oreb', 'dreb', 'reb', 'ast', 'stl', 'blk', 
            'to', 'pf', 'pts', 'plus_minus'
        ]
        clean_boxscores_df[cols_to_convert_to_int64] = clean_boxscores_df[cols_to_convert_to_int64].astype('Int64')
        
        # Convert "min" column to float
        if 'min' in clean_boxscores_df.columns:
            clean_boxscores_df['min'] = clean_boxscores_df['min'].replace('None', np.nan)
            min_split = clean_boxscores_df['min'].str.split(':', expand=True)
            clean_boxscores_df['minutes'] = pd.to_numeric(min_split[0], errors='coerce')
            clean_boxscores_df['seconds'] = pd.to_numeric(min_split[1], errors='coerce')
            clean_boxscores_df['min_float'] = clean_boxscores_df['minutes'] + (clean_boxscores_df['seconds'] / 60)
            clean_boxscores_df['min_float'] = clean_boxscores_df['min_float'].round(2)
            clean_boxscores_df['min'] = clean_boxscores_df['min_float']
            clean_boxscores_df = clean_boxscores_df.drop(['minutes', 'seconds', 'min_float'], axis=1, errors='ignore')
        else:
            logger.warning("No 'min' column in boxscores. Setting 'min' to null.")
            clean_boxscores_df['min'] = pd.nan
        
        return clean_boxscores_df
    
    except Exception as e:
        logger.error(f'Error in process_boxscores: {str(e)}')
        raise e
        
        
        
# Entry Point
event = {
    'initial_load': True
}

lambda_handler(event, {})

       team_id               team_name       nickname team_abbreviation  \
0   1610612737           Atlanta Hawks          Hawks               ATL   
1   1610612738          Boston Celtics        Celtics               BOS   
2   1610612739     Cleveland Cavaliers      Cavaliers               CLE   
3   1610612740    New Orleans Pelicans       Pelicans               NOP   
4   1610612741           Chicago Bulls          Bulls               CHI   
5   1610612742        Dallas Mavericks      Mavericks               DAL   
6   1610612743          Denver Nuggets        Nuggets               DEN   
7   1610612744   Golden State Warriors       Warriors               GSW   
8   1610612745         Houston Rockets        Rockets               HOU   
9   1610612746    Los Angeles Clippers       Clippers               LAC   
10  1610612747      Los Angeles Lakers         Lakers               LAL   
11  1610612748              Miami Heat           Heat               MIA   
12  1610612749         Mi

C:\Users\jorda\AppData\Local\Temp\ipykernel_12644\41444438.py:208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_processed.drop('is_active', axis=1, errors='ignore', inplace=True)
C:\Users\jorda\AppData\Local\Temp\ipykernel_12644\41444438.py:210: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_processed.rename(columns={


     season_id     team_id     game_id  game_date      matchup wl  min  pts  \
0        22024  1610612744  0022400338 2024-12-06  GSW vs. MIN  L  240   90   
1        22024  1610612741  0022400336 2024-12-06  CHI vs. IND  L  240  123   
2        22024  1610612754  0022400336 2024-12-06    IND @ CHI  W  241  132   
3        22024  1610612753  0022400333 2024-12-06    ORL @ PHI  L  241   94   
4        22024  1610612747  0022400334 2024-12-06    LAL @ ATL  L  265  132   
..         ...         ...         ...        ...          ... ..  ...  ...   
819      12024  1610612762  0012400002 2024-10-04  UTA vs. NZB  W  238  116   
820      12024  1610612750  0012400003 2024-10-04    MIN @ LAL  W  240  124   
821      12024  1610612747  0012400003 2024-10-04  LAL vs. MIN  L  241  107   
822      12024       15020  0012400002 2024-10-04    NZB @ UTA  L  239   87   
823      12024  1610612738  0012400001 2024-10-04    BOS @ DEN  W  240  107   

     fgm  fga  ...  fta  oreb  dreb  reb  ast  stl 

Error fetching box scores for game_id 0022400338 (attempt 1/3): HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)
Error fetching box scores for game_id 0022400338 (attempt 2/3): HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)
Error fetching box scores for game_id 0022400333 (attempt 1/3): HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)
Error fetching box scores for game_id 0022400315 (attempt 1/3): HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)
Error fetching box scores for game_id 0022400315 (attempt 2/3): HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)
Error fetching box scores for game_id 0022400318 (attempt 1/3): HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)
Error fetching box scores for game_id 0022400309 (attempt 1/3): HTTPSConnectionPool(host='stat

{'statusCode': 500,
 'body': '"Data ingestion failed: name \'boxscore_list\' is not defined"'}